In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import yfinance as yf


In [2]:

# Initialize the Alpaca API
api = tradeapi.REST('PK0LC2J1NSMU3IUAXF31', 'VpfhPftx0Wvmkn5PoOKabFcUUHrCSxQ8LF8dri3s', base_url='https://paper-api.alpaca.markets')

# Parameters for the backtest
symbol = 'AAPL'  # Replace with your target asset
start_date = '2022-01-01'
end_date = '2023-01-01'
initial_capital = 10000  # Starting capital in USD
position_size = 0.1  # Fraction of capital to allocate per trade

# Fetch historical data
def fetch_historical_data(symbol, start_date, end_date):
    barset = api.get_barset(symbol, 'day', start=start_date, end=end_date)
    df = barset[symbol].df
    df['returns'] = df['close'].pct_change()
    return df

# Simple moving average strategy (can be replaced with a more complex model)
def moving_average_strategy(df):
    df['SMA50'] = df['close'].rolling(window=50).mean()
    df['SMA200'] = df['close'].rolling(window=200).mean()
    df['signal'] = 0
    df['signal'][df['SMA50'] > df['SMA200']] = 1  # Buy signal
    df['signal'][df['SMA50'] < df['SMA200']] = -1  # Sell signal
    return df

# Backtesting function
def backtest(df, initial_capital, position_size):
    df['strategy_returns'] = df['signal'].shift(1) * df['returns']
    df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod() * initial_capital
    return df

# Fetch data and apply the strategy
df = fetch_historical_data(symbol, start_date, end_date)
df = moving_average_strategy(df)
df = backtest(df, initial_capital, position_size)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['cumulative_returns'], label='Strategy Returns')
plt.title('Backtesting Results')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (USD)')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate performance metrics
total_return = df['cumulative_returns'][-1] / initial_capital - 1
annualized_return = (1 + total_return) ** (365 / len(df)) - 1
sharpe_ratio = df['strategy_returns'].mean() / df['strategy_returns'].std() * np.sqrt(252)

print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

AttributeError: 'REST' object has no attribute 'get_barset'